## FPGA版 最初のデバッグ4データ

In [1]:
## FPGA Load
from pynq import Overlay
OL = Overlay("/home/xilinx/pynq/overlays/my_design/test_mnist_wrapper.bit")
OL.download()
XLNK = OL.processing_system7_0

## show IPs
print(OL.ip_dict.keys(), XLNK)

dict_keys(['axi_dma', 'axi_gpio_0']) <pynq.xlnk.Xlnk object at 0xafd76210>


In [2]:
## Assert i_cllear
OL.axi_gpio_0.register_map.GPIO_DATA = 0x1
OL.axi_gpio_0.register_map.GPIO_DATA = 0x0

In [3]:
IMAGE_NUM = 4
## Allocate Memory
input_buf = XLNK.cma_array([28*28*IMAGE_NUM//4])
print(hex(input_buf.physical_address))
output_buf = XLNK.cma_array([1*IMAGE_NUM//4])
print(hex(output_buf.physical_address))

0x18049000
0x1804a000


In [4]:
## Load Image
import re

def load_image(file_name, input_buf):
    with open(file_name, 'r') as f:
        val_s = 0
        for i, l in enumerate(f.readlines()):
            l = l.strip() ## remove return code
            l = re.sub("\s*\/\/.*$", "", l) ## remove Comment
            l = re.sub("\s+", " ", l) ## Remove Space 2+
            adr, val = l.split(" ")  ## split Space
            val = int(val, 16)

            val_s = val_s | (val << (8*(i%4)))
            if i%4==3:
                ## print(f"@{i//4}: 0x{val_s:08X}")
                input_buf[i//4] = val_s
                ## input_buf[i//4] = (i//4)
                val_s = 0

image_size = (28*28)//4
load_image("rom/image0.mem", input_buf[image_size*0:image_size*1]) ## 7
load_image("rom/image1.mem", input_buf[image_size*1:image_size*2]) ## 2
load_image("rom/image2.mem", input_buf[image_size*2:image_size*3]) ## 1
load_image("rom/image3.mem", input_buf[image_size*3:image_size*4]) ## 0


In [5]:

## ILAのために開始位置のデータ改変
input_buf[image_size*0] = 0x1
input_buf[image_size*1] = 0x2
input_buf[image_size*2] = 0x3
input_buf[image_size*3] = 0x4


In [6]:
# for i in range(28*28//4):
#     print(f"{i}: {input_buf[i]:08X}")

In [7]:
# input_buf.dtype

In [8]:
# ## Allocate Memory
# input_buf = OL.processing_system7_0.cma_array([16])
# print(hex(input_buf.physical_address))
# output_buf = OL.processing_system7_0.cma_array([16])
# print(hex(output_buf.physical_address))

# for i in range(16):
#     output_buf[i] = 234
#     input_buf[i] = i+16

# OL.copy_mem_0.register_map.inputBaseMemAdr_V = input_buf.physical_address
# OL.copy_mem_0.register_map.outputBaseMemAdr_V = output_buf.physical_address
# OL.copy_mem_0.register_map.loadSize_V = 12

# OL.copy_mem_0.register_map.CTRL.AP_START = 0x1

# for i in range(16):
#     print(i, input_buf[i], output_buf[i])

In [9]:
# for i in range(28*28):
#     input_buf[i] = (
#         (((i*4+3)&0xFF)<<24)|
#         (((i*4+2)&0xFF)<<16)|
#         (((i*4+1)&0xFF)<<8)|
#         (((i*4+0)&0xFF)<<0))
#     print(hex(input_buf[i])


In [10]:
## Check Output Buf
for i in range(IMAGE_NUM//4):
    output_buf[i] = 0xFFFFFFFF
print(output_buf)

[4294967295]


In [11]:
## DMA Control
## Stop
OL.axi_dma.register_map.MM2S_DMACR = 0x0
OL.axi_dma.register_map.S2MM_DMACR = 0x0

## Run
OL.axi_dma.register_map.MM2S_DMACR = 0x1
OL.axi_dma.register_map.S2MM_DMACR = 0x1

## Address
OL.axi_dma.register_map.MM2S_SA = input_buf.physical_address
OL.axi_dma.register_map.S2MM_DA = output_buf.physical_address

## Size
OL.axi_dma.register_map.MM2S_LENGTH = 28*28*4
OL.axi_dma.register_map.S2MM_LENGTH = 1*4


In [12]:
## Check Output Buf
print(f"0x{output_buf[0]:08X}")

0x00010207


In [13]:
## DMA MM2S Status
OL.axi_dma.register_map.MM2S_DMASR

Register(Halted=0, Idle=1, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SGDecErr=0, IOC_Irq=1, Dly_Irq=0, Err_Irq=0, IRQThresholdSts=0, IRQDelaySts=0)

In [31]:
## DMA S2MM Status
OL.axi_dma.register_map.S2MM_DMASR

Register(Halted=0, Idle=1, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SGDecErr=0, IOC_Irq=1, Dly_Irq=0, Err_Irq=0, IRQThresholdSts=0, IRQDelaySts=0)

In [32]:
## DMA Soft Reset
OL.axi_dma.register_map.MM2S_DMACR = 0x4
OL.axi_dma.register_map.S2MM_DMACR = 0x4
OL.axi_dma.register_map.MM2S_DMACR, OL.axi_dma.register_map.MM2S_DMACR

(Register(RS=0, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
 Register(RS=0, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0))

---

# FPGA版 1転送 Loop

In [33]:
## FPGA Load
from pynq import Overlay
OL = Overlay("/home/xilinx/pynq/overlays/my_design/test_mnist_wrapper.bit")
OL.download()
XLNK = OL.processing_system7_0

## show IPs
print(OL.ip_dict.keys(), XLNK)

dict_keys(['axi_dma', 'axi_gpio_0']) <pynq.xlnk.Xlnk object at 0xafdb6210>


In [34]:
import mnist
import numpy as np
test_x = mnist.test_images()
test_y = mnist.test_labels()

test_x = test_x.reshape(10000, 28*28)

In [35]:
## Allocate Memory
IMAGE_NUM = 1
print(IMAGE_NUM)
input_buf = XLNK.cma_array([28*28*IMAGE_NUM], np.uint8)
print(hex(input_buf.physical_address))
output_buf = XLNK.cma_array([1*IMAGE_NUM], np.uint8)
print(hex(output_buf.physical_address))

1
0x1804b000
0x1804c000


In [36]:
def copy_mem(n):
    ## Write inpu_buf(DDR)
    XLNK.cma_memcopy(input_buf, test_x[n], 28*28*1)    

def start_dma():
    ## DMA Control
    ## Stop
    OL.axi_dma.register_map.MM2S_DMACR = 0x0
    OL.axi_dma.register_map.S2MM_DMACR = 0x0

    ## Run
    OL.axi_dma.register_map.MM2S_DMACR = 0x1
    OL.axi_dma.register_map.S2MM_DMACR = 0x1

    ## Address
    OL.axi_dma.register_map.MM2S_SA = input_buf.physical_address
    OL.axi_dma.register_map.S2MM_DA = output_buf.physical_address

    ## Size
    OL.axi_dma.register_map.MM2S_LENGTH = 28*28*IMAGE_NUM
    OL.axi_dma.register_map.S2MM_LENGTH = 1*IMAGE_NUM

def wait_dma():
    i = 0
    while i<100:
        st = OL.axi_dma.register_map.S2MM_DMASR.Idle
        if st:
            break
        i += 1


In [37]:
%%time
N = 1000
ok = 0
for n in range(N):
    copy_mem(n)
    start_dma()
    wait_dma()
    if test_y[n] == output_buf[0]:
        ok += 1
    ## print(output_buf[0])
    print(f"\r{n+1}/{N}", end='')
print()
print(ok/N)

1000/1000
0.927
CPU times: user 1min 16s, sys: 12 s, total: 1min 28s
Wall time: 1min 22s


## TMP

In [14]:
import neuralnet_mnist_int
import numpy as np

In [17]:
network = neuralnet_mnist_int.init_network()
network.keys()

dict_keys(['b2', 'W1', 'b1', 'W2', 'W3', 'b3'])

In [20]:
w1 = network['W1']
w1

array([[ -1,  -2,  -3, ...,   5, -11,  -3],
       [ -2,  -4,  -3, ...,   4,   7,   3],
       [ -3,   0,  -4, ...,   2,   3,   8],
       ...,
       [  5,  -7, -16, ...,  -2,   2,   2],
       [ -1,   5,  -3, ...,   6,  -3,   6],
       [  0, -17,   2, ...,  -8,   1, -10]])

In [21]:
w2 = network['W2']
w2

array([[ -27,    4, -113, ...,    9,   35,   10],
       [  76,   -8,  -22, ...,  -12,  -24,   75],
       [  16,  161,    5, ...,   12,   68,  -33],
       ...,
       [ -47,  -27,   64, ...,   15,   29,   73],
       [  25,   23,   -9, ...,   38,  -14,   -1],
       [  -5,  -13,    1, ..., -135, -103,    2]])

In [29]:
print(w1.shape, w1.max(), w1.min(), w1.mean())
print(w2.shape, w2.max(), w2.min(), w2.mean())


(784, 50) 123 -101 0.15943877551020408
(50, 100) 219 -256 -0.2104


In [30]:
b1 = network['b1']
b2 = network['b2']

In [31]:
b2

array([ -3, -18,   0,  31,  29,  -1,  10, -21,   7,   5,  50,  11, -16,
       -13,   2,   7,  10,  17, -19,  17, -23,  14,  -9,   5,   9,  -7,
        -4,  17,  13,  17,  16,  -5,  -4,   4,   4,  -4, -18,  -4,  -5,
        -2,  11, -30,   7, -19,  -1, -22,  -4,   8,  -4, -19,  -6,  -9,
       -13,  -5,  17,   6,   0,   2,  10,  -7,  18,   0,   5,  10, -12,
         6,  15,  21,   0,  -7, -16,  12, -30,   9, -10,   8,  -8, -34,
         0,   0,  11,  -6,  -5,   5,   2,   4,  -2,  14,  12, -11,  19,
        12,  17,  10,   0,   1, -24, -13,   5, -11])